In [1]:
import os
os.getcwd()
os.chdir("../..")
os.getcwd()

'/Users/calebchiam/Documents/GitHub/Cornell-Conversational-Analysis-Toolkit'

We can use Hypergraph features for various predictive tasks:

In [2]:
import convokit
import pickle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
corpus = convokit.Corpus(filename=convokit.download("reddit-corpus"))

Dataset already exists at /Users/calebchiam/.convokit/downloads/reddit-corpus


In [4]:
hc = convokit.HyperConvo(prefix_len=10, min_thread_len=10, include_root=False)
hc.fit_transform(corpus)

In [5]:
remake_cache = False
if remake_cache:
    with open("hyperconvo_feats.p", "wb") as f:
        hyperconvo_feats = {}
        for convo in corpus.iter_conversations():
            hyperconvo_feats.update(convo.meta["hyperconvo"])
        pickle.dump(hyperconvo_feats, f)
else:
    with open("hyperconvo_feats.p", "rb") as f:
        hyperconvo_feats = pickle.load(f)

In [6]:
threads = corpus.utterance_threads(include_root=False)

In [7]:
missing = 0
present = 0
for tlc_id, thread in threads.items():
    if tlc_id not in thread:
        missing += 1
    else:
        present += 1

In [8]:
# Use only the first 10 comments in each thread
thread_pfxs = corpus.utterance_threads(prefix_len=10, include_root=False)

In [9]:
len(threads)

100000

In [10]:
from collections import defaultdict
thread_roots_by_self_post = defaultdict(list)
for top_level_comment, thread in threads.items():
    rt = thread[next(iter(thread))].root
    thread_roots_by_self_post[rt].append(top_level_comment)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
path_stats = hc.retrieve_motif_pathway_stats(corpus)

In [13]:
motif_counts = hc.retrieve_motif_counts(corpus)

In [14]:
threads_motifs = hc.retrieve_motifs(corpus)

In [15]:
threads_motifs['dnppqdj']

{'NO_EDGE_TRIADS': [<convokit.hyperconvo.triadMotif.TriadMotif at 0x1c2b34f28>,
 'SINGLE_EDGE_TRIADS': [<convokit.hyperconvo.triadMotif.TriadMotif at 0x1c2b343c8>,
 'INCOMING_TRIADS': [],
 'OUTGOING_TRIADS': [],
 'DYADIC_TRIADS': [<convokit.hyperconvo.triadMotif.TriadMotif at 0x11fb3fe80>,
 'UNIDIRECTIONAL_TRIADS': [],
 'INCOMING_2TO3_TRIADS': [],
 'INCOMING_1TO3_TRIADS': [],
 'DIRECTED_CYCLE_TRIADS': [],
 'OUTGOING_3TO1_TRIADS': [],
 'INCOMING_RECIPROCAL_TRIADS': [],
 'OUTGOING_RECIPROCAL_TRIADS': [],
 'DIRECTED_CYCLE_1TO3_TRIADS': [],
 'DIRECIPROCAL_TRIADS': [],
 'DIRECIPROCAL_2TO3_TRIADS': [],
 'TRIRECIPROCAL_TRIADS': []}

In [16]:
latent_motif_count = {thread_id: hc._latent_motif_count(motif_dict, trans=False)[0] for thread_id, motif_dict in threads_motifs.items()}

In [25]:
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import normalize, StandardScaler, Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut
from sklearn.feature_selection import f_classif, SelectPercentile

In [36]:
import random
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split

random.seed(2019)

for task in ["comment-growth", "commenter-growth"]: #, "post-deleted", "user-deleted"
    print("task {}".format(task))
    pos, neg = [], []
    for roots in thread_roots_by_self_post.values():
        has_pos, has_neg = [], []
        for root in roots:
            if task == "comment-growth":
                if len(threads[root]) >= 15:
                    has_pos.append(root)
                if len(threads[root]) == 10:
                    has_neg.append(root)
            elif task == "commenter-growth":
                if len(threads[root]) >= 20:
                    if len(set(c.user.name for c in threads[root].values())) >= \
                        len(set(c.user.name for c in thread_pfxs[root].values())) * 2:
                            has_pos.append(root)
                    else:
                        has_neg.append(root)
#             elif task == "post-deleted":
#                 if len(threads[root]) >= 10:
#                     if threads[root][root].user.info["post-deleted"]:
#                         has_pos.append(root)
#                     else:
#                         has_neg.append(root)
#             elif task == "user-deleted":
#                 if len(threads[root]) >= 10:
#                     if threads[root][root].user.info["user-deleted"]:
#                         has_pos.append(root)
#                     else:
#                         has_neg.append(root)
            else:
                print("unrecognized task name")

        if has_pos and has_neg:
            pos.append(random.choice(has_pos))
            neg.append(random.choice(has_neg))

    print("- {} positive, {} negative pts".format(len(pos), len(neg)))

    # make data from pos and neg
    X = []
    X_volume, X_reply, X_bow = [], [], []
    X_motifpath = []
    X_motifcount = []
    X_latentmotif = []
    threads_text = []
    for root in pos + neg:
        # get ordered set of feature values
        v = [hyperconvo_feats[root][k] for k in sorted(hyperconvo_feats[root].keys())]
        # data cleaning
        v = [t if (not np.isnan(t) and np.isfinite(t)) else 0 for t in v]
        X.append(v)
        
        X_motifpath.append(list(path_stats[root].values()))
        X_motifcount.append(list(motif_counts[root].values()))
        X_latentmotif.append(list(latent_motif_count[root].values()))
        # volume baseline - get num participants in thread with at least length of 10
        X_volume.append([len(set(c.user.name for c in thread_pfxs[root].values()))])   
        # reply tree baseline
        X_reply.append([hyperconvo_feats[root][k] if (not np.isnan(hyperconvo_feats[root][k]) and np.isfinite(hyperconvo_feats[root][k])) else 0 for k in sorted(hyperconvo_feats[root].keys()) 
                        if "c->c" in k])
        # BOW baseline text
        thread_text = " ".join([u.text for u in thread_pfxs[root].values()
                                if not (task == "post-deleted" and u.id == root)])  
        # don't consider root post for post-deleted task, since we could just look for the string "[deleted]"
        threads_text.append(thread_text)
        
    ys = [1]*len(pos) + [0]*len(neg)

    X, ys = np.array(X), np.array(ys)
    X_volume = np.array(X_volume)
    X_reply = np.array(X_reply)
    X_motifpath = np.array(X_motifpath)
    X_motifcount = np.array(X_motifcount)
    X_latentmotif = np.array(X_latentmotif)
    X_hc_path = np.concatenate([X_motifpath, X], axis=1)
    X_hc_count = np.concatenate([X_motifcount, X], axis=1)
    X_hc_latent = np.concatenate([X_latentmotif, X], axis=1)
    X_all = np.concatenate([X_motifpath, X_motifcount, X_latentmotif, X], axis=1)
    for X_tmp, name in [(X, "hyperconvo"), 
#                         (X_volume, "volume"), 
                        (X_reply, "reply tree"), 
                        (None, "BOW"), 
                        (X_motifpath, "motifpaths"), 
                        (X_motifcount, "motifcounts"),
                        (X_latentmotif, "latentmotifcounts"),
                        (X_hc_path, "hyperconv-motifpaths"),
                        (X_hc_count, "hyperconvo-motifcounts"),
                        (X_hc_latent, "hyperconvo-latentmotifs"),
                        (X_all, "hyperconvo-motifall")
                       ]:
        if name == "BOW":
            text_train, text_test, y_train, y_test = train_test_split(threads_text, ys, test_size=0.1, random_state=42)
            cv = CountVectorizer(min_df=0.05, max_df=0.8)
            X_train = cv.fit_transform(text_train)
            X_test = cv.transform(text_test)
        else:
            X_train, X_test, y_train, y_test = train_test_split(X_tmp, ys, test_size=0.1, random_state=42)
        
        clf = Pipeline([("normalizer", Normalizer()), ("logreg", LogisticRegression(solver='liblinear'))])
#         clf = Pipeline([("normalizer", Normalizer()), ("featselect", SelectPercentile(f_classif, 10)), ("logreg", LogisticRegression(solver='liblinear'))])
#         clf = GridSearchCV(base_clf, {"logreg__C": [10**i for i in range(-4,4)], "featselect__percentile": list(range(10, 110, 10))}, cv=3)

#         print(X_train.shape)
#         print(y_train.shape)

#         clf = LogisticRegression(solver="liblinear")
        clf.fit(X_train, y_train)

        train_acc = clf.score(X_train, y_train)
        test_acc = clf.score(X_test, y_test)
        print("- {}: {:.4f} train, {:.4f} test".format(name, train_acc, test_acc))

task comment-growth
- 1723 positive, 1723 negative pts
- hyperconvo: 0.5917 train, 0.5362 test
- reply tree: 0.5688 train, 0.5362 test
- BOW: 0.6488 train, 0.5159 test
- motifpaths: 0.5669 train, 0.5159 test
- motifcounts: 0.5643 train, 0.5101 test
- latentmotifcounts: 0.5727 train, 0.5246 test
- hyperconv-motifpaths: 0.5898 train, 0.5623 test
- hyperconvo-motifcounts: 0.5892 train, 0.5565 test
- hyperconvo-latentmotifs: 0.5850 train, 0.5478 test
- hyperconvo-motifall: 0.5859 train, 0.5536 test
task commenter-growth
- 1121 positive, 1121 negative pts
- hyperconvo: 0.5622 train, 0.5600 test
- reply tree: 0.5563 train, 0.5378 test
- BOW: 0.6728 train, 0.5111 test
- motifpaths: 0.5335 train, 0.4800 test
- motifcounts: 0.5211 train, 0.4667 test
- latentmotifcounts: 0.5206 train, 0.5156 test
- hyperconv-motifpaths: 0.5597 train, 0.5556 test
- hyperconvo-motifcounts: 0.5607 train, 0.5511 test
- hyperconvo-latentmotifs: 0.5607 train, 0.5600 test
- hyperconvo-motifall: 0.5627 train, 0.5511 tes